# **This Notebook is provided Training Example (PhC-C2DH-U373 Dataset) Using Colab**
The notebook arragemnet is as follow

*   **Prequesities**
  1.   Conect to github, Clone the repository
  2.   Configurate working directory and google drive connetction
  3.   Download and install Conda, create conda enviroment with all the required packages

*   **Trainings**
  1.   Run feature extraction for metric learning
  2.   Run metric learning training
  3.   Run compreensive feature extraction with metric learning
  4.   Run Cell Tracking training

*   **Inference**








In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Prequesities**

# 1. Conect to github, Clone the repository


In [2]:
import os
! git clone https://github.com/talbenha/cell-tracker-gnn

curr_workdir = f'./cell-tracker-gnn'
if os.path.exists(curr_workdir):
  os.chdir(curr_workdir)

Cloning into 'cell-tracker-gnn'...
remote: Enumerating objects: 504, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 504 (delta 179), reused 191 (delta 156), pack-reused 277 (from 1)
Receiving objects: 100% (504/504), 38.01 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (261/261), done.


# 2. Configurate working directory and google drive connetction


In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/CellTracking/nature_method_ctc/' # TODO: update this path
os.listdir(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Training', 'Test']

# 3. Download and install Conda, create conda enviroment with all the required packages

In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################

# try to run the bare minimum to get a new conda env working
env_name = 'cell-tracking-challenge'
conda_path = ''
try:
    conda_path = ! which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
    import sys
    sys.path.append('/usr/local/lib/python3.7/site-packages/')
    ! conda init
    ! conda install -n root _license -y -q
else:
    print('found miniconda')

conda_envs = ! conda env list
res = [i for i in conda_envs if env_name in i]
if (len(res) == 0):
    print(f'not found {env_name} env', len(res))
    ! conda create --name cell-tracking-challenge --file requirements-conda.txt
else:
    print(f'found {env_name} env', len(res))
print(f'{env_name} was created')


installing miniconda
--2024-08-22 14:54:28--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  29.4MB/s    in 2.8s    

2024-08-22 14:54:32 (29.4 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-pa

In [ ]:
%%bash
source activate cell-tracking-challenge
# install other requirements
pip install -r requirements.txt
pip install PyYAML==5.4.1

# **Training**
 The first two stages can be skipped (step #2 is long in colab) by using the pre-trained model (in step #3) provided under the [releases](https://github.com/talbenha/cell-tracker-gnn/releases) of the repository. For example for the pretrained model for PhC-C2DH-U373 dataset, download the software and find the model under `software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth`

# 1. Run feature extraction for metric learning


In [ ]:
%%bash
# Đặt các biến môi trường
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path
export SEQUENCE=01
export MODALITY="2D"
export DATASET="${FOLDER_PATH}/${DATA_NAME}"
export CODE_TRA=src/inference/
export PYTHONPATH=/content/cell-tracker-gnn

In [ ]:
# Tải pretrain model
# Tải file .zip từ liên kết
!wget -O /content/software.zip https://github.com/talbenha/cell-tracker-gnn/releases/download/CTC/software.zip

# Giải nén file .zip vào thư mục /content/software
!unzip /content/software.zip -d /content/software

# Kiểm tra nội dung thư mục đã giải nén
!ls /content/software

In [ ]:
# Đường dẫn đến mô hình pretrained đã giải nén
export MODEL_PYTORCH_LIGHTNING='/content/software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth' # TODO: update this path

# Kiểm tra các đường dẫn
if [ ! -d "${DATASET}" ]; then
  echo "Directory ${DATASET} does not exist"
  exit 1
fi

# Dự đoán: Chạy model để dự đoán quãng đường đi của các tế bào
python ${CODE_TRA}/inference_clean.py -mp "${MODEL_PYTORCH_LIGHTNING}" -ns "${SEQUENCE}" -oc "${DATASET}"

# Hậu xử lý: Tạo bản đồ nhãn từ các dự đoán
python ${CODE_TRA}/postprocess_clean.py -modality "${MODALITY}" -iseg "${DATASET}/${SEQUENCE}_GT/TRA" -oi "${DATASET}/${SEQUENCE}_RES_inference"

# 3. Run compreensive feature extraction with metric learning

In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export METRIC_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc/software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path

python run_feat_extract.py params.input_images="${FOLDER_PATH}/${DATA_NAME}" params.input_masks="${FOLDER_PATH}/${DATA_NAME}" params.input_seg="${FOLDER_PATH}/${DATA_NAME}" params.output_csv="${FOLDER_PATH}/ct_features/" params.sequences=['01','02']  params.seg_dir='_GT/TRA' params.basic=False params.input_model="${METRIC_PATH}"

# 4. Run Cell Tracking training

In [ ]:
%%bash
source activate cell-tracking-challenge
export HYDRA_FULL_ERROR=1
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path
# cell tracking training run
python run.py datamodule.dataset_params.main_path="${FOLDER_PATH}/ct_features/${DATA_NAME}" datamodule.dataset_params.exp_name="2D_SIM" datamodule.dataset_params.drop_feat=[]

# **Inference**
Update the following variables:


*   MODEL_PYTORCH_LIGHTNING - according to the output of the previius cell
*   MODEL_METRIC_LEARNING
*   FOLDER_PATH - path to folder that contains the dataset
*   MODALITY





In [ ]:
%%bash
source activate cell-tracking-challenge
export SEQUENCE=01 # sequence number
export FOLDER_PATH=/content/drive/MyDrive/CellTracking/nature_method_ctc # TODO: update this path
export MODEL_METRIC_LEARNING=/content/drive/MyDrive/CellTracking/nature_method_ctc/software/parameters/Features_Models/PhC-C2DH-U373/all_params.pth # TODO: update this path
export MODEL_PYTORCH_LIGHTNING='/content/cell-tracker-gnn/logs/runs/2022-07-16/19-37-07/checkpoints/epoch=37.ckpt'
export DATA_NAME=PhC-C2DH-U373 # TODO: update this path
export MODALITY="2D"  # dataset modality

export DATASET="${FOLDER_PATH}/${DATA_NAME}"
export CODE_TRA=src/inference/
export PYTHONPATH=/content/cell-tracker-gnn

# our model needs CSVs, so let's create from image and segmentation.
python ${CODE_TRA}/preprocess_seq2graph_clean.py -cs 20 -ii "${DATASET}/${SEQUENCE}" -iseg "${DATASET}/${SEQUENCE}_GT/TRA" -im "${MODEL_METRIC_LEARNING}" -oc "${DATASET}/${SEQUENCE}_CSV"

# run the prediction
python ${CODE_TRA}/inference_clean.py -mp "${MODEL_PYTORCH_LIGHTNING}" -ns "${SEQUENCE}" -oc "${DATASET}"

# create label maps
python ${CODE_TRA}/postprocess_clean.py -modality "${MODALITY}" -iseg "${DATASET}/${SEQUENCE}_GT/TRA" -oi "${DATASET}/${SEQUENCE}_RES_inference"
